# Working with Spans

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [2]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [3]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [4]:
instruments.Instrument().print_instrumentation()

Module examples.expositional.end2end_apps.custom_app.custom_app*
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomApp
    Method retrieve_chunks: (self, data)
    Method respond_to_query: (self, input)
    Method arespond_to_query: (self, input)
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomTemplate
    Method fill: (self, question, answer)

Module examples.expositional.end2end_apps.custom_app.custom_llm*
  Class examples.expositional.end2end_apps.custom_app.custom_llm.CustomLLM
    Method generate: (self, prompt: str)

Module examples.expositional.end2end_apps.custom_app.custom_memory*
  Class examples.expositional.end2end_apps.custom_app.custom_memory.CustomMemory
    Method remember: (self, data: str)

Module examples.expositional.end2end_apps.custom_app.custom_retriever*
  Class examples.expositional.end2end_apps.custom_app.custom_retriever.CustomRetriever
    Method retrieve_chunks: (self, data)
      Span type: SpanType.RETRIEVER

Mo

In [5]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()

In [6]:
rec.calls[0].model_dump()

{'stack': [{'path': 'app',
   'method': {'obj': {'cls': {'name': 'CustomApp',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_app'},
      'bases': None},
     'id': 11462560208,
     'init_bindings': None},
    'name': 'respond_to_query'}},
  {'path': 'app',
   'method': {'obj': {'cls': {'name': 'CustomApp',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_app'},
      'bases': None},
     'id': 11462560208,
     'init_bindings': None},
    'name': 'retrieve_chunks'}},
  {'path': 'app.retriever',
   'method': {'obj': {'cls': {'name': 'CustomRetriever',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_retriever'},
      'bases': None},
     'id': 11476781776,
    

In [7]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame([(s.trace_id, s.name, s.span_type, s.span_id, s.parent_span_id, s.attributes) for s in spans])


,0,1,2,3,4,5
0,340282366920938463461285262137769617781,root,SpanRoot,4370756423165289941,NaN,"{'trulens_eval@span_type': 'SpanRoot', 'trulen..."
1,340282366920938463461285262137769617781,retrieve_chunks,SpanRetriever,4154731841429800986,9.875346e+18,"{'trulens_eval@span_type': 'SpanRetriever', 't..."
2,340282366920938463461285262137769617781,retrieve_chunks,SpanOther,9875345988340903741,3.375420e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
3,340282366920938463461285262137769617781,remember,SpanOther,13508494306138764491,3.375420e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
4,340282366920938463461285262137769617781,generate,SpanOther,267610227878581883,3.375420e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
5,340282366920938463461285262137769617781,fill,SpanOther,5987148653028257066,3.375420e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
6,340282366920938463461285262137769617781,remember,SpanOther,18191205197263462402,3.375420e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
7,340282366920938463461285262137769617781,respond_to_query,SpanOther,3375420497167857037,4.370756e+18,"{'trulens_eval@span_type': 'SpanOther', 'trule..."


In [8]:
for span in spans:
    pprint(span)
    pprint(span.model_dump())
    print()

SpanRoot(name='root', kind=<SpanKind.INTERNAL: 0>, status=<StatusCode.OK: 1>, status_description=None, start_timestamp=1714181527594077000, end_timestamp=1714156327458285000, context=HashableSpanContext(trace_id=0xffffffffffffffffe301278662146975, span_id=0x3ca80bcf428625d5, trace_flags=0x00, trace_state=[], is_remote=False), events=[], links={}, attributes={'trulens_eval@span_type': 'SpanRoot', 'trulens_eval@record_id': 'record_hash_b9a1b716f15720549fa790baca2f08ac'}, attributes_metadata={}, record=Record(record_id='record_hash_b9a1b716f15720549fa790baca2f08ac', app_id='customapp', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 4, 26, 18, 32, 6, 641802), end_time=datetime.datetime(2024, 4, 26, 18, 32, 7, 458285)), ts=datetime.datetime(2024, 4, 26, 18, 32, 7, 458338), tags='-', meta=None, main_input='hello', main_output="The answer to hello is 

/opt/homebrew/Caskroom/miniconda/base/envs/py311_trulens/lib/python3.11/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `Union[str, bool, int, float, json-or-python[json=list[str], python=list[str]], json-or-python[json=list[bool], python=list[bool]], json-or-python[json=list[int], python=list[int]], json-or-python[json=list[float], python=list[float]]]` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
